In [1]:
import os
import sys
sys.path.append('common')
import util, audio_preprocessing

import shutil
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint, Callback, TensorBoard
from matplotlib import pyplot as plt
%matplotlib inline
from cyclic_lr import CyclicLR

from dataset import SingleDataset, MultiDataset
from model import model_mlt_cnn_alexnet

TRY = 'MIX7'
BASE = 'MIX4'
LOG_DIR = './%slog' % TRY
SCD_PREFIX = 'scd_'
USD_PREFIX = 'usd_'

batch_size = 32
epochs = 20

scd = SingleDataset(SCD_PREFIX,
                   util.load_txt_list(os.path.join('.', SCD_PREFIX+'classes.txt')),
                   batch_size)
usd = SingleDataset(USD_PREFIX,
                   ['air_conditioner',  'car_horn', 'children_playing', 'dog_bark', 'drilling',
                    'engine_idling', 'gun_shot','jackhammer', 'siren', 'street_music'],
                   batch_size)

Using TensorFlow backend.


In [2]:
# Clean up log
shutil.rmtree(LOG_DIR, ignore_errors=True)

In [3]:
# Fine tune for scd
multi = MultiDataset([scd, usd], [1.0, 0.0])

model = model_mlt_cnn_alexnet(multi.input_shape(), multi.ys_classes(), freeze_mode=1)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.load_weights('model%s.h5' % BASE) ### Starting point

callbacks = [
    CyclicLR(base_lr=0.00007, max_lr=0.0007, step_size=multi.train_steps_per_epoch, mode='triangular'),
    ModelCheckpoint('model%s.h5' % TRY,
                monitor='val_output_1_acc',
                verbose=1,
                save_best_only=True,
                save_weights_only=True),
     keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
]
model.fit_generator(multi.train_generator,
                    steps_per_epoch=multi.train_steps_per_epoch,
                    epochs=epochs,
                    validation_data=multi.valid_generator, 
                    validation_steps=multi.valid_steps_per_epoch,
                    callbacks=callbacks)

multi.evaluate_by_datasets(model)

# Fine tune for usd
multi = MultiDataset([scd, usd], [0.0, 1.0])

model = model_mlt_cnn_alexnet(multi.input_shape(), multi.ys_classes(), freeze_mode=2)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.load_weights('model%s.h5' % TRY)

callbacks = [
    CyclicLR(base_lr=0.00007, max_lr=0.0007, step_size=multi.train_steps_per_epoch, mode='triangular'),
    ModelCheckpoint('model%s.h5' % TRY,
                monitor='val_output_2_acc',
                verbose=1,
                save_best_only=True,
                save_weights_only=True),
     keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
]
model.fit_generator(multi.train_generator,
                    steps_per_epoch=multi.train_steps_per_epoch,
                    epochs=epochs,
                    validation_data=multi.valid_generator, 
                    validation_steps=multi.valid_steps_per_epoch,
                    callbacks=callbacks)

multi.evaluate_by_datasets(model)

model_mlt_cnn_alexnet: freeze other than output_1.
Epoch 1/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.8473 - output_1_loss: 1.8675 - output_2_loss: -0.0202 - output_1_acc: 0.6884 - output_2_acc: 0.1358
Epoch 00001: val_output_1_acc improved from -inf to 0.90152, saving model to modelMIX7.h5
1596/1596 [==============================] - 72s 45ms/step - loss: 1.8476 - output_1_loss: 1.8677 - output_2_loss: -0.0200 - output_1_acc: 0.6884 - output_2_acc: 0.1357 - val_loss: 0.3758 - val_output_1_loss: 0.3757 - val_output_2_loss: 9.7660e-05 - val_output_1_acc: 0.9015 - val_output_2_acc: 0.0896
Epoch 2/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.8680 - output_1_loss: 1.8710 - output_2_loss: -0.0029 - output_1_acc: 0.6920 - output_2_acc: 0.1371
Epoch 00002: val_output_1_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.8682 - output_1_loss: 1.8711 - output_2_loss: -0.0029 - output_1_acc: 0.6920 - output_2_acc: 0

Epoch 17/20
1594/1596 [============================>.] - ETA: 0s - loss: 1.8275 - output_1_loss: 1.8410 - output_2_loss: -0.0134 - output_1_acc: 0.6990 - output_2_acc: 0.1366
Epoch 00017: val_output_1_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.8273 - output_1_loss: 1.8409 - output_2_loss: -0.0136 - output_1_acc: 0.6991 - output_2_acc: 0.1366 - val_loss: 0.3698 - val_output_1_loss: 0.3736 - val_output_2_loss: -0.0038 - val_output_1_acc: 0.9007 - val_output_2_acc: 0.0868
Epoch 18/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.8444 - output_1_loss: 1.8507 - output_2_loss: -0.0063 - output_1_acc: 0.6972 - output_2_acc: 0.1388
Epoch 00018: val_output_1_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.8447 - output_1_loss: 1.8509 - output_2_loss: -0.0062 - output_1_acc: 0.6972 - output_2_acc: 0.1388 - val_loss: 0.3469 - val_output_1_loss: 0.3612 - val_output_2_loss: -0.0143 - val_output_1

Epoch 12/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.1861 - output_1_loss: -0.0276 - output_2_loss: 1.2138 - output_1_acc: 0.0406 - output_2_acc: 0.7348
Epoch 00012: val_output_2_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.1866 - output_1_loss: -0.0274 - output_2_loss: 1.2140 - output_1_acc: 0.0406 - output_2_acc: 0.7348 - val_loss: 1.5731 - val_output_1_loss: 0.0107 - val_output_2_loss: 1.5624 - val_output_1_acc: 0.0313 - val_output_2_acc: 0.5885
Epoch 13/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.2023 - output_1_loss: -0.0140 - output_2_loss: 1.2163 - output_1_acc: 0.0411 - output_2_acc: 0.7346
Epoch 00013: val_output_2_acc did not improve
1596/1596 [==============================] - 72s 45ms/step - loss: 1.2023 - output_1_loss: -0.0139 - output_2_loss: 1.2162 - output_1_acc: 0.0411 - output_2_acc: 0.7345 - val_loss: 1.5491 - val_output_1_loss: -0.0018 - val_output_2_loss: 1.5509 - val_output_1_

[0.9049720506031186, 0.5797979797979798]

## Evaluate per dataset

In [4]:
model.load_weights('model%s.h5' % TRY)
multi.evaluate_by_datasets(model)

Evaluating scd_dataset
6798/6798 [==============================] - 10s 2ms/step
 = 0.9049720506031186
Evaluating usd_dataset
990/990 [==============================] - 1s 1ms/step
 = 0.591919191919192
Accuracy with scd_dataset = 0.904972
Accuracy with usd_dataset = 0.591919


[0.9049720506031186, 0.591919191919192]

## Evaluate by mixed dataset

In [5]:
model.load_weights('model%s.h5' % TRY)
multi = MultiDataset([scd, usd], [0.5, 0.5])
results = model.evaluate_generator(multi.valid_generator, steps=multi.valid_steps_per_epoch)
print(model.metrics_names, results)

['loss', 'output_1_loss', 'output_2_loss', 'output_1_acc', 'output_2_acc'] [1.1464911148910668, 0.36378889193606617, 0.7827022227502722, 0.8215202876219826, 0.5174627632254751]
